<a href="https://colab.research.google.com/github/Vika0408/Mental-Health/blob/main/helpingminds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kagglehub

In [6]:
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("zaber666/meld-dataset")

print("Path to dataset files:", path)


Resuming download from 509607936 bytes (11295054176 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/zaber666/meld-dataset?dataset_version_number=1 (509607936/11804662112) bytes left.


100%|██████████| 11.0G/11.0G [02:02<00:00, 91.8MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1


In [7]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout, Bidirectional, MultiHeadAttention,
    LayerNormalization, GlobalAveragePooling1D
)
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.callbacks import EarlyStopping

In [8]:
DATASET_PATH = "/root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1"


In [9]:
df = pd.read_csv('/root/.cache/kagglehub/datasets/zaber666/meld-dataset/versions/1/MELD-RAW/MELD.Raw/train/train_sent_emo.csv')

In [10]:
#text preprocessing
MAX_VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, oov_token="<OOV>")
tokenizer.fit_on_texts(df['Utterance'])

sequences = tokenizer.texts_to_sequences(df['Utterance'])
padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post")


In [11]:
#label encoding
label_encoder_sentiment = LabelEncoder()
label_encoder_emotion = LabelEncoder()

df['Sentiment'] = label_encoder_sentiment.fit_transform(df['Sentiment'])
df['Emotion'] = label_encoder_emotion.fit_transform(df['Emotion'])

sentiment_labels = np.array(df['Sentiment'])
emotion_labels = np.array(df['Emotion'])

In [12]:
#split data
X_train, X_test, y_train_sentiment, y_test_sentiment, y_train_emotion, y_test_emotion = train_test_split(
    padded_sequences, sentiment_labels, emotion_labels, test_size=0.2, random_state=42
)

# Ensure proper NumPy format
y_train_sentiment = np.array(y_train_sentiment, dtype=np.int32)
y_test_sentiment = np.array(y_test_sentiment, dtype=np.int32)
y_train_emotion = np.array(y_train_emotion, dtype=np.int32)
y_test_emotion = np.array(y_test_emotion, dtype=np.int32)

In [19]:
#model define
VOCAB_SIZE = MAX_VOCAB_SIZE
EMBEDDING_DIM = 128
LSTM_UNITS = 256  # Reduced from 512 to prevent overfitting
DROPOUT_RATE = 0.3

# Input layer
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,))

# Embedding layer with masking
x = Embedding(VOCAB_SIZE, EMBEDDING_DIM, mask_zero=True)(inputs)

# Bidirectional LSTM with dropout
x = Bidirectional(LSTM(LSTM_UNITS, return_sequences=True, dropout=DROPOUT_RATE, recurrent_dropout=0.2, unroll=False, use_cudnn=False))(x)

padded_sequences = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding="post", truncating="post")


# Multi-Head Self-Attention
attention_output = MultiHeadAttention(num_heads=4, key_dim=LSTM_UNITS)(x, x)
x = LayerNormalization()(x + attention_output)

# Global Pooling
x = GlobalAveragePooling1D()(x)

# Sentiment output
sentiment_output = Dense(3, activation="softmax", name="sentiment_output")(x)

# Emotion output
emotion_output = Dense(7, activation="softmax", name="emotion_output")(x)

# Build Model
model = Model(inputs, outputs=[sentiment_output, emotion_output])

# Compile Model
model.compile(
    optimizer=AdamW(learning_rate=0.0003, weight_decay=1e-5),
    loss={
        "sentiment_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        "emotion_output": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    },
    metrics={
        "sentiment_output": ["accuracy"],
        "emotion_output": ["accuracy"]
    }
)



In [20]:
# Model Summary
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 100, 128)       │      1,280,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal_2 (NotEqual)    │ (None, 100)            │              0 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 100, 512)       │        788,480 │ embedding_2[0][0],     │
│ (Bidirectional)           │                        │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 100, 512)       │      2,100,736 │ bidirectional_2[0][0], │
│ (MultiHeadAttention)      │                        │                │ bidirectional_2[0][0], │
│                           │                        │                │ not_equal_2[0][0],     │
│                           │                        │                │ not_equal_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 100, 512)       │              0 │ bidirectional_2[0][0], │
│                           │                        │                │ multi_head_attention_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 100, 512)       │          1,024 │ add_2[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_average_pooling1d… │ (None, 512)            │              0 │ layer_normalization_2… │
│ (GlobalAveragePooling1D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sentiment_output (Dense)  │ (None, 3)              │          1,539 │ global_average_poolin… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ emotion_output (Dense)    │ (None, 7)              │          3,591 │ global_average_poolin… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 4,175,370 (15.93 MB)

 Trainable params: 4,175,370 (15.93 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

history = model.fit(
    X_train,
    {"sentiment_output": y_train_sentiment, "emotion_output": y_train_emotion},  # ✅ Corrected output names
    epochs=30,
    batch_size=64,
    validation_data=(X_test, {"sentiment_output": y_test_sentiment, "emotion_output": y_test_emotion}),
    callbacks=[early_stopping]
)



Epoch 1/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 111s 805ms/step - emotion_output_accuracy: 0.4484 - emotion_output_loss: 1.5846 - loss: 2.6458 - sentiment_output_accuracy: 0.4579 - sentiment_output_loss: 1.0611 - val_emotion_output_accuracy: 0.5030 - val_emotion_output_loss: 1.4441 - val_loss: 2.4349 - val_sentiment_output_accuracy: 0.5345 - val_sentiment_output_loss: 1.0105
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 806ms/step - emotion_output_accuracy: 0.4907 - emotion_output_loss: 1.4023 - loss: 2.3632 - sentiment_output_accuracy: 0.5451 - sentiment_output_loss: 0.9609 - val_emotion_output_accuracy: 0.5135 - val_emotion_output_loss: 1.4351 - val_loss: 2.4097 - val_sentiment_output_accuracy: 0.5591 - val_sentiment_output_loss: 0.9919
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 142s 803ms/step - emotion_output_accuracy: 0.5634 - emotion_output_loss: 1.2386 - loss: 2.0795 - sentiment_output_accuracy: 0.6213 - sentiment_output_loss: 0.8409 - val_emotion_output_accuracy: 0.5200 - val_emotion_outpu